This is a detailed walkthrough for analyzing pySCENIC + scRNA-seq data. It is intended to be ran without much modification to the code, however, the file architecture will be different depending on use case. In addition, cistarget databases and transcription factor databases will vary as well. Consider the goal of the analysis and use this workflow as a guideline.



**TLDR:** Use this as a guideline. For the most part, variables declared '...' are marked for user input.

In [ ]:
# Import modules
import pandas as pd
import numpy as np
import os, glob
import pickle
import sys
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
import louvain
import leidenalg
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell
import loompy
import seaborn as sns
import matplotlib.gridspec


The file architecture will be dependent on use case. However, it's important to keep note of what files are required for this analysis:

1. Single cell expression data (either .csv or .loom) 
2. A csv of adjecensies from the pySCENIC GRN step.
3. A csv of regulons from the pySCENIC ctx step.
4. A file of known TFs for target organism. One TF per line of the .txt file. (http://bioinfo.life.hust.edu.cn/AnimalTFDB4/#/Download) 
5. From cisTarget database you'll need a genome ranking database (.feather) and motif annotation database (https://resources.aertslab.org/cistarget/) 
    -  I take two .feather files , one for centered region around TSS and one for upstream of TSS. cisTarget has instructions for navigating the database. 

In [ ]:
# Define File architecture
PROJECT_FOLDER="..."
DATABASE_FOLDER = "..."
INPUT_FOLDER = "..."
ADJACENCIES_FOLDER = "..."

#Change to where your output files are from pyscenic cli steps
REGULONS_FOLDER = "..."
AUCELL_FOLDER = "..."

DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "...")
MOTIF_ANNOTATIONS_FNAME = os.path.join(DATABASE_FOLDER, "...")

MM_TFS_FNAME = os.path.join(DATABASE_FOLDER, '...')
SC_EXP_FNAME = os.path.join(INPUT_FOLDER, "...")

ADJACENCIES_FNAME = os.path.join(ADJACENCIES_FOLDER, "...")
REGULONS_FNAME = os.path.join(REGULONS_FOLDER, "...")

# If running aucell on CLI, define location below. Currently, the pipeline runs aucell within itself as it doesn't take very long. [OPTIONAL]
AUC_FNAME = os.path.join(AUCELL_FOLDER, "...")


In [ ]:
# Start with the expression data info from the input loom file 
ex_matrix = anndata.read_loom(SC_EXP_FNAME)

# Convert to a pd dataframe
ex_matrix = anndata.AnnData.to_df(ex_matrix)

# Visualize the data, it should have cells as rows and genes as columns. Also check dimensions for data loss. 
ex_matrix

First, we'll do some initial processing to create an adata object and prep it for later integration and analysis.

In [ ]:
# Validate expression dataframe
def is_valid_exp_matrix(mtx):
    return (all(isinstance(idx, str) for idx in mtx.index) 
            and all(isinstance(idx, str) for idx in mtx.columns)
            and (mtx.index.nlevels == 1)
            and (mtx.columns.nlevels == 1))
is_valid_exp_matrix(ex_matrix)

In [ ]:
# Scanpy Settings can be changed given user preference.
sc.settings.verbosity = 3 
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80,facecolor='white')

In [ ]:
# Convert to adata for processing
adata = sc.AnnData(ex_matrix)
gene_symbols = ex_matrix.columns

#Set the varname to gene_symbols
adata.var['gene_symbols'] = gene_symbols
adata.var_names_make_unique()
adata # returns shape (cellsXgenes)

**NOTE:** At this point, we begin pySCENIC steps. For this workflow, we run the first two steps of pySCENIC (grn, and ctx) via CLI commands submitted as an sbatch script. See **run_grn_nodask.sh** and **run_ctx.sh**.

 The first slurm script for inferring the gene regulatory network has an approximate runtime of 24-28 hours per 10,000 cells with a depth of approximately 6,000 Genes/Cell, given the default resources allocated in the configuration. The second step, ctx, doesn't take as long - however, we ran each step except for aucell (3rd and final step) as sbatch jobs. 

 **NOTE:** If your single cell expression matrix is sparse (likely), then you should opt for the '--sparse' argument for the GRN step, it significantly reduces the runtime of this step. 


The GRN step will produce a csv of adjacensies. Consisting of transcription factors, and corresponding target genes. The importance column is a measure of impact. The higher the importance of a transcription factor to the target gene, the more regulative power it has over it. This is simply an overlapping of the genes present in your given expression matrix, and known transcription factors for your target organism.

In [ ]:
# Read in the adjacencies file created from the grn step. 
adjacencies = pd.read_csv(ADJACENCIES_FNAME, index_col=False, sep=',')
adjacencies.head()

Next, we examine the output from the ctx step, which performs motif enrichment on the transcription factors in our data. This creates profiles of genes which are targetted by transcription factors, quantifying the importance with parameters like AUC and motifsimilarity. We will use this information to compile regulons within the data. By using this information, we generate 'regulons' , consisting of genes which regulate one another surrounding a central target gene. The regulon object generated below consists of target genes, and weights per transcription factor. This will act as the input for the final step, aucell. 

In [ ]:
# Regulon csv needed some reformatting in my case to deal with multi-indexing with TF/MotifID
from pyscenic.utils import load_motifs
import operator as op
from IPython.display import HTML, display

BASE_URL = "http://motifcollections.aertslab.org/v9/logos/"
COLUMN_NAME_LOGO = "MotifLogo"
COLUMN_NAME_MOTIF_ID = "MotifID"
COLUMN_NAME_TARGETS = "TargetGenes"

def display_logos(df: pd.DataFrame, top_target_genes: int = 3, base_url: str = BASE_URL):
    """
    :param df:
    :param base_url:
    """
    # Make sure the original dataframe is not altered.
    df = df.copy()
    
    # Add column with URLs to sequence logo.
    def create_url(motif_id):
        return '<img src="{}{}.png" style="max-height:124px;"></img>'.format(base_url, motif_id)
    df[("Enrichment", COLUMN_NAME_LOGO)] = list(map(create_url, df.index.get_level_values(COLUMN_NAME_MOTIF_ID)))
    
    # Truncate TargetGenes.
    def truncate(col_val):
        return sorted(col_val, key=op.itemgetter(1))[:top_target_genes]
    df[("Enrichment", COLUMN_NAME_TARGETS)] = list(map(truncate, df[("Enrichment", COLUMN_NAME_TARGETS)]))
    
    MAX_COL_WIDTH = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', 200)
    display(HTML(df.head().to_html(escape=False)))
    pd.set_option('display.max_colwidth', MAX_COL_WIDTH)

reg = pd.read_csv(REGULONS_FNAME, sep=',',index_col=False,header=1)
reg = reg.rename(columns={'Unnamed: 0': 'TF','Unnamed: 1':'MotifID'})
reg = reg.drop(0)
reg = reg.set_index(['TF','MotifID'])
reg['TargetGenes'] = reg['TargetGenes'].astype('object')
reg.head()



In [ ]:
# Check the datatypes for each column, if any of the objects are labeled as str, convert to objects. This is just a sanity check.
print(reg.dtypes)

In [ ]:
# Add motif metadata to the dataframe 
df_motifs = load_motifs(REGULONS_FNAME)

# If you want, you can examine data for a specific transcription factor.
selected_motifs = ['...']
df_motifs_sel = df_motifs.iloc[ [ True if x in selected_motifs else False for x in df_motifs.index.get_level_values('TF') ] ,:]
display_logos( df_motifs_sel.sort_values([('Enrichment','NES')], ascending=False))


In [ ]:
# Convert the dataframe to a sequence of regulons. 
regulons = df2regulons(df_motifs)

For the last step of pySCENIC, we run AUCell. This will calculate regulon module scores for each cell in our data. Essentially, this measures regulon activity per cell so we can get an idea of the regulatory systems interacting within it. Think of it as a metric of regulon enrichment. The higher the AUCell metric, the more enriched a regulon is within the cell. 

In [ ]:
# Run aucell, this may take a few minutes. For 50,000 cells and 30,000 genes it took ~  5 minutes
auc_mtx = aucell(ex_matrix, regulons, num_workers=1)

In [ ]:
# Visualize the Aucell Matrix. This shows regulon enrichment per cell across all cells/regulons identified in your data. Rows should be cells, columns should be regulons.
auc_mtx

We will also binarize this matrix to represent regulon activation per cell. This step uses the binarize function from pySCENIC , and does a per regulon calculation of bimodal distributions to set an automated custom threshold to determine what 'activated' vs. 'deactivated' looks like in terms of AUCell scores. 

In [ ]:
# Binarization function fits a bimodal distribution per regulon across all of its enrichment scores, using this as a threshold for determining activation state. This may take a few minutes.
from pyscenic.binarization import binarize
bin_auc = binarize(auc_mtx=auc_mtx, num_workers=20)

In [ ]:
# If the binarized matrix doesn't resemble the structure of the non-binarized (Which resembles a pandas dataframe) , run the following one-liner. 
#bin_auc = bin_auc[0]
bin_auc

In [ ]:
# We can subset the auc matrices based on the experimental conditions we're dealing with.
# NOTE: I provide an example one liner of how this is done, use it as a template for the conditions you're dealing with. 

# Subset by generic condition
Ctrl_auc = auc_mtx[auc_mtx.index.str.contains('Ctrl')]
Ctrl_bin_auc = bin_auc[bin_auc.index.str.contains('Ctrl')]

stim_auc = auc_mtx[auc_mtx.index.str.contains('stim')]
stim_bin_auc = bin_auc[bin_auc.index.str.contains('stim')]

In [ ]:
# Let's briefly visualize the normal and binarized AUCell matrices via clustermaps. These will hierarchically cluster cells based on regulon enrichment or activation. (Cells on Y axis, regulons on X axis)
sns.clustermap(bin_auc,figsize=(12,12))
sns.clustermap(auc_mtx, figsize=(12,12))

In [ ]:
average_enrichment = auc_mtx.mean().to_dict()

plt.figure(figsize=(20,8))
plt.hist(average_enrichment.keys(), weights=average_enrichment.values())
plt.xticks(rotation = 45)
plt.title('Average Regulon Enrichment Across All Cells')
plt.tight_layout()

In [ ]:
# Combining several subset dictionaries into a dataframe for visualization.
average_enrichment_1 = auc_mtx[auc_mtx.index.str.contains('Day1')].mean().to_dict()
average_enrichment_2 = auc_mtx[auc_mtx.index.str.contains('Day2')].mean().to_dict()
average_enrichment_3 = auc_mtx[auc_mtx.index.str.contains('Day3')].mean().to_dict()
dicts = [average_enrichment_1,average_enrichment_2,average_enrichment_3]

avg_enrichment_by_age = pd.DataFrame.from_dict(dicts).transpose()
new_column_names = ['Day1','Day2','Day3']
avg_enrichment_by_age.columns = new_column_names

In [ ]:
avg_enrichment_by_age.plot(kind='bar', stacked=True,figsize=(20,8))
plt.xticks(rotation = 45)
plt.title('Average Regulon Enrichment Over 3 Days')
plt.tight_layout()


In [ ]:
# Line graph depiction of enrichment change over time, given a regulon of interest.
plt.figure(figsize=(23,8))
plt.plot(avg_enrichment_by_age[['REGULON OF INTEREST']])

For this next section, we will integrate the pySCENIC AUC matrix data with our initial adata object containing single cell expression data, along with any relevant metadata that we wish to investigate. In the case of this workflow, there are three primary variables. (age, odor exposure, and genotype) This may differ depending on use-case, so this will be one of the few code-chunks that will likely be changed. I also recommend at least having the "source" variable if you are working with a merged object. In this case, it is likely your cell IDs have been tagged with their sample metadata, if the data was initially merged as a Seurat object. We rely on this cell ID nomenclature to extract this information on a per-cell basis, ensuring accuracy. 

NOTE: If this is not the case in a particular dataset, the index 'grep' strategy employed below will not work. 

In [ ]:
# Initialize empty metadata columns
adata.obs['age'] = None


In [ ]:
# Create obs columns for experimental variables
def obs_from_str(data, string, colname):
    'Look for particular string in the index (Cell IDs) of adata, and assigns string to corresponding value in obs column'
    matches = data.obs.index.str.contains(string)
    data.obs.loc[matches,colname] = string
    return data

obs_from_str(adata,'Day1',colname='age')
obs_from_str(adata,'Day2',colname='age')
obs_from_str(adata,'Day3',colname='age')


adata.obs

In [ ]:
# You can subset TF motif table based on TF of interest. This is the data used to generate regulon object later on.
selected_motifs = ['TF OF INTEREST']
df_motifs_sel = df_motifs.iloc[ [ True if x in selected_motifs else False for x in df_motifs.index.get_level_values('TF') ] ,:]
display_logos( df_motifs_sel.sort_values([('Enrichment','NES')], ascending=False))

In [ ]:
tf = 'TF OF INTEREST'

#Basically doing ctx step of pySCENIC CLI. Doesn't take too long, however.  
modules = list(modules_from_adjacencies(adjacencies,ex_matrix))

In [ ]:
def find_index(regulon_list, regulon):
    'Find the index of regulons list that has the desired regulon of interest'
    found_index = -1
    for i, entry in enumerate(regulon_list):
        if regulon in str(entry):
            found_index == i
    return found_index
   

tf_mods = [ x for x in modules if x.transcription_factor==tf ]
find_index(regulon_list=regulons, regulon="REGULON OF INTEREST")

In [ ]:
#Figure out where the regulon is in the regulons object
for i,entry in enumerate(regulons):
    if regulons[i].name == 'REGULON OF INTEREST': # Change this depending on regulon of interest
        print(f'REGULON OF INTEREST regulon is at index {i}')
        index = i 
#Extract how many genes are in the regulon, and modules.
for i,mod in enumerate( tf_mods ):
    print( f'{tf} module {str(i)}: {len(mod.genes)} genes' )
print( f'{tf} regulon: {len(regulons[index])} genes' )




In [ ]:
# We can extract this information to a text file for further analysis.
for i,mod in enumerate( tf_mods ):
    with open( tf+'_module_'+str(i)+'.txt', 'w') as f:
        for item in mod.genes:
            f.write("%s\n" % item)

with open( tf+'_regulon.txt', 'w') as f:
    for item in regulons[index].gene2weight:
        f.write("%s\n" % item)

The remaining workflow will be heavily custom depending on the goal of the analysis. However, a good first step for this analysis is to integrate your pySCENIC data with your adata object from Scanpy. This will be necessary in most cases, allowing you to see regulon enrichment across cell types, project enrichment in UMAP space, see regulon specificity per cell type, and much more. Here is some example code.

In [ ]:
# Integrating pySCENIC data with adata.
from pyscenic.export import add_scenic_metadata

add_scenic_metadata(adata, auc_mtx, regulons)

#NOTE: You can rerun this to add the binarized auc_mtx as an additional layer. In my case the binary regulon columns are indicated with a '_y' following the name of the regulon. This may not always be the case!

In [ ]:
# Regulon Specificity scores by celltype. Must have annotated cells.
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_rss
from adjustText import adjust_text

rss_cellType = regulon_specificity_scores(auc_mtx=auc_mtx, cell_type_series=adata.obs['celltype'])

cats = sorted(list(set(adata.obs['celltype'])))

fig = plt.figure(figsize=(15, 8))
for c,num in zip(cats, range(1,len(cats)+1)):
    x=rss_cellType.T[c]
    ax = fig.add_subplot(3,5,num)
    plot_rss(rss_cellType, c, top_n=5, max_n=None, ax=ax)
    ax.set_ylim( x.min()-(x.max()-x.min())*0.05 , x.max()+(x.max()-x.min())*0.05 )
    for t in ax.texts:
        t.set_fontsize(12)
    ax.set_ylabel('')
    ax.set_xlabel('')
    adjust_text(ax.texts, autoalign='xy', ha='right', va='bottom', arrowprops=dict(arrowstyle='-',color='lightgrey'), precision=0.001 )
 
fig.text(0.5, 0.0, 'Top Regulons per CellType: All Cells', ha='center', va='center', size='x-large')
fig.text(0.00, 0.5, 'Regulon specificity score (RSS)', ha='center', va='center', rotation='vertical', size='x-large')
plt.tight_layout()
plt.rcParams.update({
    'figure.autolayout': True,
        'figure.titlesize': 'large' ,
        'axes.labelsize': 'medium',
        'axes.titlesize':'large',
        'xtick.labelsize':'medium',
        'ytick.labelsize':'medium'
        })
plt.show()

In [ ]:
# Subsetting the auc mtx based on top regulons per cell type, and visualizing this in a clustermap
topreg = []
for i,c in enumerate(cats):
    topreg.extend(
        list(rss_cellType.T[c].sort_values(ascending=False)[:5].index)
    )
topreg = list(set(topreg))

auc_mtx_Z = pd.DataFrame( index=auc_mtx.index )
for col in list(auc_mtx.columns):
    auc_mtx_Z[ col ] = ( auc_mtx[col] - auc_mtx[col].mean()) / auc_mtx[col].std(ddof=0)

In [ ]:
# Setting up the colormap for celltypes on y axis.
def palplot(pal, names, colors=None, size=1):
    n = len(pal)
    f, ax = plt.subplots(1, 1, figsize=(n * size, size))
    ax.imshow(np.arange(n).reshape(1, n),
              cmap=mpl.colors.ListedColormap(list(pal)),
              interpolation="nearest", aspect="auto")
    ax.set_xticks(np.arange(n) - .5)
    ax.set_yticks([-.5, .5])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    colors = n * ['k'] if colors is None else colors
    for idx, (name, color) in enumerate(zip(names, colors)):
        ax.text(0.0+idx, 0.0, name, color=color, horizontalalignment='center', verticalalignment='center')
    return f
colors = sns.color_palette('bright',n_colors=len(cats) )
colorsd = dict( zip( cats, colors ))
colormap = [ colorsd[x] for x in adata.obs['celltype'] ]

sns.set()
sns.set(font_scale=0.8)
fig = palplot( colors, cats, size=1.0)

handles = [Patch(facecolor=colorsd[name]) for name in colorsd]

In [ ]:
# Plotting the clustermap
sns.set(font_scale=1.2)
g = sns.clustermap(auc_mtx_Z[topreg], annot=False,  square=False,  linecolor='gray',
    yticklabels=False, xticklabels=True, vmin=-2, vmax=6, row_colors=colormap,
    cmap="YlGnBu", figsize=(21,16) )
g.cax.set_visible(True)
g.ax_heatmap.set_ylabel('')
g.ax_heatmap.set_xlabel('')
plt.legend(handles,colorsd,title ='Cell Type', bbox_to_anchor =(1, 1), bbox_transform=plt.gcf().transFigure,loc = 'center left')
plt.show()

In [ ]:
# Plotting Enrichment in UMAP space (Uses nonbinarized aucell matrix in integrated adata)
sc.pl.umap(adata=adata,color='Regulon(REGULON)',ax= axs[1,0], show=False, color_map='plasma',title="Regulon of interest Enrichment Across all Cells")

# Plotting activation in UMAP space (Uses binarized aucell matrix in integrated adata)
sc.pl.umap(adata=adata,color='Regulon(REGULON)_y',ax= axs[1,1],show=False,color_map='Spectral_r', title="Regulon of interest Activation Across all Cells")

# NOTE: The activation umap implies the '_y' nomenclature mentioned prior, if yours is different then change accordingly. 

**Further Reading:** This workflow is based off a combination of established workflows provided by SCENIC/pySCENIC authors. Check them out if you want a starting place to potentially improve this downstream workflow.

- https://htmlpreview.github.io/?https://github.com/aertslab/SCENICprotocol/blob/master/notebooks/PBMC10k_downstream-analysis.html
- https://github.com/aertslab/SCENICprotocol/blob/master/notebooks/SCENIC%20Protocol%20-%20Case%20study%20-%20Mouse%20brain%20data%20set.ipynb
- https://github.com/hbc/knowledgebase/blob/master/scrnaseq/pySCENIC.md (For CLI guidance)
